#  **Identificando discurso de ódio pelo ChatGPT-4o**

**Autores:** Luiz Fernando de Aguiar Lima, Hérique Costa Ribeiro de Lima, Adonias Caetano de Oliveira

## **Installation**

In [ ]:
# Uninstall any existing openai installations
#!pip uninstall -y openai
# Install the required version of openai
#!pip install openai==0.28
!pip install openai

## **Importing libraries**

In [ ]:
import os
import openai
import pandas as pd
import re
import random
import os
import asyncio
from openai import AsyncOpenAI

## **Loading data**

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1lOKAxCtQ7v0q0cL_IfxhLDKCMbDtbV6B/edit?usp=sharing&ouid=107874559379957908991&rtpof=true&sd=true'
file_id = url.split('/')[-2]
read_url='https://drive.google.com/uc?id=' + file_id

# read the data
dataset = pd.read_excel(read_url, index_col=0)
dataset.head()

,tweet,class
0,"@jennybaquing so terms like honkey, cracker, p...",0
1,I think dat @Miss_Facetious called me a nigger...,0
2,RT @UNCLERUCKUS_BD: Being a nigger isn't a col...,0
3,"@NYRKelsMads @DaniloNavarro85 Hey fag, I said ...",0
4,RT @DefendWallSt: ...wow? RT @1SleepingGiant: ...,0


## **ChatGPT model**

In [ ]:
OPENAI_API_KEY="<INSIRA A CHAVE API AQUI>"


In [ ]:
client = AsyncOpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

## **Classificação das amostras**

In [ ]:
from google.colab import files

def save_respostas(respostas_llm):
  df = pd.DataFrame(respostas_llm)
  df.to_excel(f'respostas_chatgpt.xlsx', index=False)

  # Especificar o caminho do arquivo no Google Drive
  files.download(f'respostas_chatgpt.xlsx')

In [ ]:
async def main() -> None:
  respostas_llm = {}

  sentencas = dataset["tweet"].to_list()
  targets = dataset["class"].to_list()

  respostas_llm["text"] = sentencas
  respostas_llm["target"] = targets
  respostas_llm['predicted'] = []

  for sentenca in sentencas:

    prompt = f"""
    Você é um modelo de IA treinado para classificar frases como:

    - "0" se a frase contém discurso de ódio (ataques direcionados a grupos com base em raça, religião, orientação sexual, etc.).
    - "1" se a frase contém linguagem ofensiva, mas não for discurso de ódio.
    - "2" se a frase não contém discurso de ódio nem linguagem ofensiva

    Regras:
    - Responda *apenas* com "0", "1" ou "2".
    - Não adicione explicações ou qualquer outro texto.

    Frase: "{sentenca}"
    """
    response = await client.chat.completions.create(
              model="gpt-4o",
              messages=[{"role": "user", "content": prompt}]
    )

    # Access the content using dot notation instead of square brackets
    respostas_llm['predicted'].append(response.choices[0].message.content)

  save_respostas(respostas_llm)

# Use await directly instead of asyncio.run()
await main()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>